In [33]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext

from credmark.dto import EmptyInput

from credmark.cmf.types import Contract, Token, Price, Address, Account, Contracts, Tokens, Accounts

from credmark.cmf.types.ledger import (
    BlockTable, ContractTable,
    LogTable, ReceiptTable, TokenTable, TokenTransferTable,
    TraceTable, TransactionTable, LedgerTable,
    LedgerAggregate, LedgerModelOutput
)

from credmark.cmf.engine.dev_models.console import get_dt, get_block

In [34]:
import logging

logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        level=logging.DEBUG)

# Initialize

In [35]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://localhost:10444'},
          'api_url': None,
          'run_id': None,
          'console': True,
          'use_local_models': '*'
         }
context = EngineModelContext.create_context(**params)

# model_loader.loaded_model_version_lists()
# model_loader.reload()

ledger = context.ledger
run_model = context.run_model
models = context.models
block_number = context.block_number
chain_id = context.chain_id
web3 = context.web3
run_model_historical = context.historical.run_model_historical
run_model_historical_blocks = context.historical.run_model_historical_blocks

## Use Cmf

### 1. Basic

In [20]:
get_dt(2022, 5, 3)

datetime.datetime(2022, 5, 3, 0, 0, tzinfo=datetime.timezone.utc)

In [21]:
get_block(get_dt(2022, 5, 3))

14701368

In [22]:
context.block_number

14891502

In [23]:
context.chain_id

1

In [24]:
context.web3

In [25]:
context.web3.eth.get_balance('0xd3CdA913deB6f67967B99D67aCDFa1712C293601')

1790191567590102228

In [26]:
context.web3.eth.get_block(context.block_number)['gasUsed']

16041303

### 2. Use models

1. Type `models.` and hit `TAB` key, a list of models will be pop up.
2. Select a model and type ? will show the help, `models.token.info?`
3. Show the list of models wiht `model_loader.loaded_model_versions()`
4. If you have updated any model in the folder, `model_loader.reload()`.

In [27]:
models.token.info?

Call signature:
models.token.info(
    input: Union[pydantic.main.BaseModel, dict, NoneType] = None,
    return_type: Union[dict, Type[pydantic.main.BaseModel], NoneType] = None,
    **kwargs,
) -> Union[dict, pydantic.main.BaseModel]
Type:           RunModelMethod
String form:    <credmark.cmf.model.context.RunModelMethod object at 0x7f5d2c6bf820>
File:           ~/mambaforge/envs/cmk310/lib/python3.10/site-packages/credmark_model_framework-0.8.22-py3.10.egg/credmark/cmf/model/context.py
Docstring:     
token.info
- slug: token.info
- displayName: Token Information
- description: Return token's information
- latestVersion: 1.1
- developer: Credmark
- input schema (* for required field):
  Token(Token(*))
    └─address(string)
- input example:
  #01: {"address": "0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9"}
  #02: {"symbol": "AAVE"}
  #03: {"address": "0x1F98431c8aD98523631AE4a59f267346ea31F984"}
  #04: {"address": "0x1F98431c8aD98523631AE4a59f267346ea31F984", "abi": "(Optional) contra

### 3. Create types

In [28]:
usdt = Token(address='0xdAC17F958D2ee523a2206206994597C13D831ec7')
print(usdt.decimals)
context.run_model('chainlink.price-usd', input=usdt)

6


{'price': 0.9995,
 'src': 'chainlink.price-by-registry|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|True|t:9533s|r:0'}

In [29]:
context.run_model('chainlink.price-usd', input=usdt, block_number=get_block(get_dt(2022, 5, 3)))

{'price': 0.9995,
 'src': 'chainlink.price-by-registry|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|True|t:9533s|r:0'}

#### 3.1 Run a model as of current block or in the past - BUG - same result

In [30]:
get_block(get_dt(2022, 5, 3))

14701368

In [31]:
aave = Token(symbol='AAVE')

print(block_number.timestamp_datetime,
      context.models.chainlink.price_usd(input=aave, return_type=Price).price,
      context.run_model('chainlink.price-usd', input=aave, return_type=Price).price)

for dt in [get_dt(2022, 5, 3), get_dt(2022, 5, 30), get_dt(2022, 1, 1)]:
    block = get_block(dt)
    print(block.timestamp_datetime,
      context.models(block).chainlink.price_usd(input=aave, return_type=Price).price,
      context.run_model('chainlink.price-usd', input=aave, return_type=Price, block_number=block).price)

2022-06-02 14:26:55+00:00 105.59356164 105.59356164
2022-05-02 23:59:50+00:00 105.59356164 105.59356164
2022-05-29 23:59:55+00:00 105.59356164 105.59356164
2021-12-31 23:59:49+00:00 105.59356164 105.59356164


#### 3.2 Travel between different blocks

One-way only - can only travel to smaller block numbers, and not back.

In [16]:
context.block_number = get_block(get_dt(2022, 5, 3))
models.chainlink.price_usd(input=aave, return_type=Price).price

144.94887195

In [17]:
# Below will gave any error as we wants to travel back
# context.block_number = get_block(get_dt(2022, 5, 30))
# models.chainlink.price_usd(input=aave, return_type=Price).price

### 4. Ledger

In [36]:
(context.ledger.get_transactions(columns=[TransactionTable.Columns.BLOCK_HASH,
                                          TransactionTable.Columns.FROM_ADDRESS,
                                          TransactionTable.Columns.TO_ADDRESS,
                                          TransactionTable.Columns.VALUE],
                                 where=f'{TransactionTable.Columns.BLOCK_NUMBER} = 14861063',
                                 order_by=TransactionTable.Columns.BLOCK_TIMESTAMP,
                                 limit='5')
 .to_dataframe())

,block_hash,from_address,to_address,value
0,0xb7bf14ba8e4101a59a2116dd225e780593f823edf2ab...,0x4aa042f28ae6e3ee714d8e4435dc7110f7e3c3c7,0x3dd98c8a089dbcff7e8fc8d4f532bd493501ab7f,0
1,0xb7bf14ba8e4101a59a2116dd225e780593f823edf2ab...,0x30cb2c51fc4f031fa5f326d334e1f5da00e19ab5,0x1111111254fb6c44bac0bed2854e76f90643097d,0
2,0xb7bf14ba8e4101a59a2116dd225e780593f823edf2ab...,0xeb55a78c79b91719f6817855c5ad43a7aa0bf08c,0xdac17f958d2ee523a2206206994597c13d831ec7,0
3,0xb7bf14ba8e4101a59a2116dd225e780593f823edf2ab...,0x1840fa4d4e24fae28fcdf24157ea9af119fe999e,0x0039a4124f3917f4a93747d0ce43434a10fc70bb,213155603590621150
4,0xb7bf14ba8e4101a59a2116dd225e780593f823edf2ab...,0xa7efae728d2936e78bda97dc267687568dd593f3,0x268eaa7fdc3e879d11380176c4c1f16f5b25d999,198930000000000000
